In [78]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

forex_df = dd.read_csv("/Users/robertwillfindyou/Downloads/HISTDATA_COM_ASCII_GBPUSD_T202501/DAT_ASCII_GBPUSD_T_202501.csv", header=None, blocksize=None)

forex_df.columns = ['datetime', 'bid', 'ask', 'other']

forex_df.head(10)

,datetime,bid,ask,other
0,20250101 170123324,1.25134,1.25240,0
1,20250101 170134857,1.25134,1.25236,0
2,20250101 170138819,1.25138,1.25236,0
3,20250101 170205545,1.25136,1.25236,0
4,20250101 170420239,1.25136,1.25236,0
5,20250101 170432938,1.25136,1.25218,0
6,20250101 170433241,1.25136,1.25209,0
7,20250101 170436259,1.25136,1.25218,0
8,20250101 170436511,1.25136,1.25214,0
9,20250101 170436763,1.25136,1.25209,0


In [79]:
print(forex_df.bid.describe().compute())

count    2.660109e+06
mean     1.234478e+00
std      1.137918e-02
min      1.209920e+00
25%      1.223650e+00
50%      1.234950e+00
75%      1.244150e+00
max      1.257560e+00
Name: bid, dtype: float64


In [80]:
forex_df["price"] = (forex_df["bid"] + forex_df["ask"])/2
forex_df.head()

,datetime,bid,ask,other,price
0,20250101 170123324,1.25134,1.25240,0,1.25187
1,20250101 170134857,1.25134,1.25236,0,1.25185
2,20250101 170138819,1.25138,1.25236,0,1.25187
3,20250101 170205545,1.25136,1.25236,0,1.25186
4,20250101 170420239,1.25136,1.25236,0,1.25186


In [ ]:


# Intrinsic Time Transformation Function
def intrinsic_time_transformation(tick_data, delta_up, delta_down):
    """
    Apply intrinsic time transformation to tick data.
    
    Parameters:
    - tick_data (pd.DataFrame): DataFrame containing 'timestamp' and 'price' columns.
    - delta_up (float): Threshold for significant upward price movements.
    - delta_down (float): Threshold for significant downward price movements.
    
    Returns:
    - events (pd.DataFrame): DataFrame with identified intrinsic events.
    """
    # Initialize variables
    events = []
    mode = None  # Market direction (None at start)
    first_tick = True
    S_ext = None  # Extreme price
    S_IE = None   # Last intrinsic event price
    
    # Iterate through tick data
    for index, row in tick_data.iterrows():
        timestamp, S_tick = row['timestamp'], row['price']
        
        # Handle first tick
        if first_tick:
            S_ext = S_tick
            S_IE = S_tick
            first_tick = False
            mode = 'up'  # Initial mode assumption
            continue

        if mode == 'up':
            if S_tick - S_ext >= delta_up:
                mode = 'down'
                S_ext = S_tick
                S_IE = S_tick
                events.append((timestamp, S_tick, 1))  # Uptrend event
            elif S_tick < S_ext:
                S_ext = S_tick
                if S_IE - S_ext >= delta_down:
                    S_IE = S_tick
                    events.append((timestamp, S_tick, -2))  # Downtrend confirmation
        elif mode == 'down':
            if S_ext - S_tick >= delta_down:
                mode = 'up'
                S_ext = S_tick
                S_IE = S_tick
                events.append((timestamp, S_tick, -1))  # Downtrend event
            elif S_tick > S_ext:
                S_ext = S_tick
                if S_ext - S_IE >= delta_up:
                    S_IE = S_tick
                    events.append((timestamp, S_tick, 2))  # Uptrend confirmation

    # Convert to DataFrame
    events_df = pd.DataFrame(events, columns=['timestamp', 'price', 'event_type'])
    return events_df

# Sample tick data (replace with actual data)
data = {
    'timestamp': pd.date_range(start='2025-01-01 09:00:00', periods=100, freq='S'),
    'price': 100 + (pd.Series(range(100)) * 0.02)  # Simulated price data
}
df = pd.DataFrame(data)

# Define thresholds
delta_up = 0.1
delta_down = 0.1

# Apply intrinsic time transformation
intrinsic_events = intrinsic_time_transformation(df, delta_up, delta_down)

# Display results
print(intrinsic_events)


In [ ]:
import dask.dataframe as dd
import pandas as pd

# Function to apply intrinsic time transformation to a pandas DataFrame partition
def intrinsic_time_transformation_partition(df, delta_up, delta_down):
    """
    Apply intrinsic time transformation to a pandas DataFrame partition.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'datetime', 'bid', and 'ask' columns.
    - delta_up (float): Threshold for significant upward price movements.
    - delta_down (float): Threshold for significant downward price movements.

    Returns:
    - pd.DataFrame: DataFrame with an additional 'event_type' column.
    """
    # Calculate mid-price
    df['mid_price'] = (df['bid'] + df['ask']) / 2

    # Initialize variables
    events = []
    mode = None  # Market direction (None at start)
    first_tick = True
    S_ext = None  # Extreme price
    S_IE = None   # Last intrinsic event price

    # Iterate through tick data
    for index, row in df.iterrows():
        timestamp, S_tick = row['datetime'], row['mid_price']

        # Handle first tick
        if first_tick:
            S_ext = S_tick
            S_IE = S_tick
            first_tick = False
            mode = 'up'  # Initial mode assumption
            events.append(0)  # No event for the first tick

        elif mode == 'up':
            if S_tick - S_ext >= delta_up:
                mode = 'down'
                S_ext = S_tick
                S_IE = S_tick
                events.append(1)  # Uptrend reversal → Downtrend starts
            elif S_tick < S_ext:
                S_ext = S_tick
                if S_IE - S_ext >= delta_down:
                    S_IE = S_tick
                    events.append(-2)  # Downtrend confirmation
                else:
                    events.append(0)  # No event
            
        elif mode == 'down':
            if S_ext - S_tick >= delta_down:
                mode = 'up'
                S_ext = S_tick
                S_IE = S_tick
                events.append(-1)  # Downtrend reversal → Uptrend starts
            elif S_tick > S_ext:
                S_ext = S_tick
                if S_ext - S_IE >= delta_up:
                    S_IE = S_tick
                    events.append(2)  # Uptrend confirmation
                else:
                    events.append(0)  # No event
            

    # Add event types as a new column
    df['event_type'] = events
    return df.drop(columns=['mid_price'])


# Convert pandas DataFrame to Dask DataFrame with 2 partitions
#ddf = dd.from_pandas(pdf, npartitions=2)

# Define thresholds
delta_up = 0.0001
delta_down = 0.0001

# Apply the intrinsic time transformation to each partition
ddf_transformed = forex_df.map_partitions(intrinsic_time_transformation_partition, delta_up, delta_down)

# Compute the result
result = ddf_transformed.compute()

# Display results
print(result.head(100))  # Show the first 20 rows



ValueError: Metadata inference failed in `intrinsic_time_transformation_partition`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
ValueError('Length of values (1) does not match length of index (2)')

Traceback:
---------
  File "/opt/anaconda3/lib/python3.12/site-packages/dask/dataframe/utils.py", line 193, in raise_on_meta_error
    yield
  File "/opt/anaconda3/lib/python3.12/site-packages/dask_expr/_expr.py", line 4062, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9y/pjl381bj33qg4q5hcl7lwdzr0000gn/T/ipykernel_39369/1284794852.py", line 69, in intrinsic_time_transformation_partition
    df['event_type'] = ev

In [77]:
result.head(60)

,datetime,bid,ask,other,price,event_type
0,20250101 170123324,1.25134,1.25240,0,1.251870,0
1,20250101 170134857,1.25134,1.25236,0,1.251850,0
2,20250101 170138819,1.25138,1.25236,0,1.251870,0
3,20250101 170205545,1.25136,1.25236,0,1.251860,0
4,20250101 170420239,1.25136,1.25236,0,1.251860,0
5,20250101 170432938,1.25136,1.25218,0,1.251770,0
6,20250101 170433241,1.25136,1.25209,0,1.251725,-2
7,20250101 170436259,1.25136,1.25218,0,1.251770,0
8,20250101 170436511,1.25136,1.25214,0,1.251750,0
9,20250101 170436763,1.25136,1.25209,0,1.251725,0


In [26]:
print(ddf_transformed.event_type.describe().compute())

count    2.660109e+06
mean     7.894413e-06
std      2.892575e-01
min     -2.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.000000e+00
Name: event_type, dtype: float64
